In [1]:
!pip install dgl -f https://data.dgl.ai/wheels/repo.html

Looking in links: https://data.dgl.ai/wheels/repo.html
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 52.2 MB/s eta 0:00:00


In [2]:
!pip install dglgo -f https://data.dgl.ai/wheels-test/repo.html

Looking in links: https://data.dgl.ai/wheels-test/repo.html
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.5/63.5 kB 2.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 91.2/91.2 kB 4.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.2/45.2 kB 5.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.4/52.4 kB 5.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 112.2/112.2 kB 13.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.8/78.8 kB 10.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 29.4/29.4 MB 50.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.3/41.3 kB 4.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 75.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 485.6/485.6 kB 51.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 572.7/572.7 kB 49.3 MB/s eta 0:00:00
  Created wheel

In [3]:
%matplotlib inline
import os

os.environ["DGLBACKEND"] = "pytorch"
import dgl
import numpy as np
import networkx as nx
import torch
import torch.nn as nn
import dgl.function as fn
import torch.nn.functional as F
import shutil
from torch.utils.data import DataLoader
import cloudpickle
from dgl.nn import GraphConv

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
!mkdir BBBP_dataset

In [6]:
!unzip /content/drive/MyDrive/graph_dataset.zip -d BBBP_dataset

Archive:  /content/drive/MyDrive/graph_dataset.zip
  inflating: BBBP_dataset/scaffold_0_smiles_train.pickle  
  inflating: BBBP_dataset/scaffold_0_test.bin  
  inflating: BBBP_dataset/scaffold_0_val.bin  
  inflating: BBBP_dataset/scaffold_0_smiles_val.pickle  
  inflating: BBBP_dataset/scaffold_0_smiles_test.pickle  
  inflating: BBBP_dataset/scaffold_0_train.bin  


In [7]:
current_dir = "./"
checkpoint_path = current_dir + "save_models/model_checkpoints/" + "checkpoint"
os.makedirs(checkpoint_path, exist_ok=True)

best_model_path = current_dir + "save_models/best_model/"

folder_data_temp = current_dir +"data_temp/"
shutil.rmtree(folder_data_temp, ignore_errors=True)

#### Custom PyTorch Datasets

In [8]:
""" Classification Dataset """
class DGLDatasetClass(torch.utils.data.Dataset):
    def __init__(self, address):
          self.address=address+".bin"
          self.list_graphs, train_labels_masks_globals = dgl.load_graphs(self.address)
          num_graphs =len(self.list_graphs)
          self.labels = train_labels_masks_globals["labels"].view(num_graphs,-1)
          self.masks = train_labels_masks_globals["masks"].view(num_graphs,-1)
          self.globals = train_labels_masks_globals["globals"].view(num_graphs,-1)
    def __len__(self):
        return len(self.list_graphs)
    def __getitem__(self, idx):
        return  self.list_graphs[idx], self.labels[idx], self.masks[idx], self.globals[idx]

#### Defining Train, Validation and Test sets

In [9]:
path_data_temp =  'BBBP_dataset/scaffold_0'
train_set = DGLDatasetClass(address=path_data_temp+"_train")
val_set = DGLDatasetClass(address=path_data_temp+"_val")
test_set = DGLDatasetClass(address=path_data_temp+"_test")

print('Train set size: ', len(train_set))
print('Validation set size: ', len(val_set))
print('Test set size: ', len(test_set))

Train set size:  1631
Validation set size:  203
Test set size:  205


#### DataLoader

In [10]:
def collate(batch):
    # batch is a list of tuples (graphs, labels, masks, globals)
    # Concatenate a sequence of graphs
    graphs = [e[0] for e in batch]
    g = dgl.batch(graphs)

    # Concatenate a sequence of tensors (labels) along a new dimension
    labels = [e[1] for e in batch]
    labels = torch.stack(labels, 0)

    # Concatenate a sequence of tensors (masks) along a new dimension
    masks = [e[2] for e in batch]
    masks = torch.stack(masks, 0)

    # Concatenate a sequence of tensors (globals) along a new dimension
    globals = [e[3] for e in batch]
    globals = torch.stack(globals, 0)

    return g, labels, masks, globals


def loader(batch_size=64):
    train_dataloader = DataLoader(train_set,
                              batch_size=batch_size,
                              collate_fn=collate,
                              drop_last=False,
                              shuffle=True,
                              num_workers=1)

    val_dataloader =  DataLoader(val_set,
                             batch_size=batch_size,
                             collate_fn=collate,
                             drop_last=False,
                             shuffle=False,
                             num_workers=1)

    test_dataloader = DataLoader(test_set,
                             batch_size=batch_size,
                             collate_fn=collate,
                             drop_last=False,
                             shuffle=False,
                             num_workers=1)
    return train_dataloader, val_dataloader, test_dataloader

In [11]:
train_dataloader, val_dataloader, test_dataloader = loader(batch_size=64)

In [12]:
g, _, _, _ = next(iter(train_dataloader))
g

Graph(num_nodes=551, num_edges=1008,
      ndata_schemes={'v': Scheme(shape=(128,), dtype=torch.float32)}
      edata_schemes={'e': Scheme(shape=(13,), dtype=torch.float32)})

## GNN

#### Some variables

In [13]:
#BBBP dataset has 1 task. Some other datasets may have some more number of tasks, e.g., tox21 has 12 tasks.
num_tasks = 1

# Size of global feature of each graph
global_size = 200

# Number of epochs to train the model
num_epochs = 100

# Number of steps to wait if the model performance on the validation set does not improve
patience = 10

#Configurations to instantiate the model
config = {"node_feature_size":127, "edge_feature_size":12, "hidden_size":100}


In [14]:
config.get('node_feature_size')

127

#### GCN with two convolutional layer.

In [ ]:
class GCN1(nn.Module):
    def __init__(self, config, global_size = 200, num_tasks = 1):
        super().__init__()
        self.config = config
        self.num_tasks = num_tasks

        # Node feature size
        self.node_feature_size = self.config.get('node_feature_size', 127)

        # Edge feature size
        self.edge_feature_size = self.config.get('edge_feature_size', 12)

        # Hidden size
        self.hidden_size = self.config.get('hidden_size', 100)

        self.conv1 = GraphConv(self.node_feature_size, self.hidden_size, allow_zero_in_degree='True')
        self.conv2 = GraphConv(self.hidden_size, self.num_tasks, allow_zero_in_degree='True')

    # def forward(self, g, in_feat):
    def forward(self, mol_dgl_graph, globals):
        mol_dgl_graph.ndata["v"]= mol_dgl_graph.ndata["v"][:,:self.node_feature_size]
        mol_dgl_graph.edata["e"] = mol_dgl_graph.edata["e"][:,:self.edge_feature_size]
        h = self.conv1(mol_dgl_graph, mol_dgl_graph.ndata["v"])
        h = F.relu(h)
        h = self.conv2(mol_dgl_graph, h)
        mol_dgl_graph.ndata["h"] = h

        return dgl.mean_nodes(mol_dgl_graph, "h")

#### Function in order to computing score(ROC-AUC)

In [15]:
from sklearn.metrics import roc_auc_score

def compute_score(model, data_loader, val_size, num_tasks):
    model.eval()
    metric = roc_auc_score
    with torch.no_grad():
        prediction_all= torch.empty(0)
        labels_all= torch.empty(0)
        masks_all= torch.empty(0)
        for i, (mol_dgl_graph, labels, masks, globals) in enumerate(data_loader):
            prediction = model(mol_dgl_graph, globals)
            prediction = torch.sigmoid(prediction)
            prediction_all = torch.cat((prediction_all, prediction), 0)
            labels_all = torch.cat((labels_all, labels), 0)
            masks_all = torch.cat((masks_all, masks), 0)
        average = torch.tensor([0.])
        for i in range(num_tasks):
            a1 = prediction_all[:, i][masks_all[:,i]==1]
            a2 = labels_all[:, i][masks_all[:,i]==1]
            try:
                t = metric(a2.int().cpu(), a1.cpu()).item()
            except ValueError:
                t = 0
            average += t
    return average.item()/num_tasks

#### Loss Function

In [16]:
def loss_func(output, label, mask, num_tasks):
    pos_weight = torch.ones((1, num_tasks))
    pos_weight
    criterion = torch.nn.BCEWithLogitsLoss(reduction='none', pos_weight=pos_weight)
    loss = mask*criterion(output,label)
    loss = loss.sum() / mask.sum()
    return loss

#### Training

In [17]:
# Training function

def train_epoch(train_dataloader, model, optimizer):
    epoch_train_loss = 0
    iterations = 0
    model.train() # Prepare model for training
    for i, (mol_dgl_graph, labels, masks, globals) in enumerate(train_dataloader):
        prediction = model(mol_dgl_graph, globals)
        loss_train = loss_func(prediction, labels, masks, num_tasks)
        optimizer.zero_grad(set_to_none=True)
        loss_train.backward()
        optimizer.step()
        epoch_train_loss += loss_train.detach().item()
        iterations += 1
    epoch_train_loss /= iterations

    return epoch_train_loss


def train_evaluate(model):
    optimizer = torch.optim.Adam(model.parameters(), lr = 0.0001)

    best_val = 0
    patience_count = 1
    epoch = 1

    while epoch <= num_epochs:
        if patience_count <= patience:
            model.train()
            loss_train = train_epoch(train_dataloader, model, optimizer)
            model.eval()
            score_val = compute_score(model, val_dataloader, len(val_set), num_tasks)
            if score_val > best_val:
                best_val = score_val
                print("Save checkpoint")
                path = os.path.join(checkpoint_path, 'checkpoint.pth')
                dict_checkpoint = {"score_val": score_val}
                dict_checkpoint.update({"model_state_dict": model.state_dict(), "optimizer_state": optimizer.state_dict()})
                with open(path, "wb") as outputfile:
                    cloudpickle.dump(dict_checkpoint, outputfile)
                patience_count = 1
            else:
                print("Patience", patience_count)
                patience_count += 1

            print("Epoch: {}/{} | Training Loss: {:.3f} | Valid Score: {:.3f}".format(
            epoch, num_epochs, loss_train, score_val))

            print(" ")
            print("Epoch: {}/{} | Best Valid Score Until Now: {:.3f}".format(epoch, num_epochs, best_val), "\n")
        epoch += 1

    # best model save
    shutil.rmtree(best_model_path, ignore_errors=True)
    shutil.copytree(checkpoint_path, best_model_path)

    print("Final results:")
    print("Average Valid Score: {:.3f}".format(np.mean(best_val)), "\n")


##### Function to compute test set score of the final saved model

In [18]:
import time
start_time = time.time()

def test_evaluate(model):
    path = os.path.join(best_model_path, 'checkpoint.pth')
    with open(path, 'rb') as f:
        checkpoint = cloudpickle.load(f)
    model.load_state_dict(checkpoint["model_state_dict"])
    model.eval()
    test_score = compute_score(model, test_dataloader, len(test_set), num_tasks)

    print("Test Score: {:.3f}".format(test_score), "\n")
    print("Execution time: {:.3f} seconds".format(time.time() - start_time))


##### Train the model and evaluate its performance

In [ ]:
model_1 = GCN1(config, global_size, num_tasks)

In [ ]:
train_evaluate(model_1)
test_evaluate(model_1)

Save checkpoint
Epoch: 1/100 | Training Loss: 0.621 | Valid Score: 0.287
 
Epoch: 1/100 | Best Valid Score Until Now: 0.287 

Save checkpoint
Epoch: 2/100 | Training Loss: 0.609 | Valid Score: 0.292
 
Epoch: 2/100 | Best Valid Score Until Now: 0.292 

Save checkpoint
Epoch: 3/100 | Training Loss: 0.602 | Valid Score: 0.303
 
Epoch: 3/100 | Best Valid Score Until Now: 0.303 

Save checkpoint
Epoch: 4/100 | Training Loss: 0.596 | Valid Score: 0.315
 
Epoch: 4/100 | Best Valid Score Until Now: 0.315 

Save checkpoint
Epoch: 5/100 | Training Loss: 0.590 | Valid Score: 0.329
 
Epoch: 5/100 | Best Valid Score Until Now: 0.329 

Save checkpoint
Epoch: 6/100 | Training Loss: 0.588 | Valid Score: 0.346
 
Epoch: 6/100 | Best Valid Score Until Now: 0.346 

Save checkpoint
Epoch: 7/100 | Training Loss: 0.584 | Valid Score: 0.361
 
Epoch: 7/100 | Best Valid Score Until Now: 0.361 

Save checkpoint
Epoch: 8/100 | Training Loss: 0.578 | Valid Score: 0.377
 
Epoch: 8/100 | Best Valid Score Until Now: 

### GCN with 4 convolutional layers and batch normalization layers and dropouts.

In [ ]:
class GCN2(nn.Module):
    def __init__(self, config, global_size = 200, num_tasks = 1):
        super().__init__()
        self.config = config
        self.num_tasks = num_tasks

        # Node feature size
        self.node_feature_size = self.config.get('node_feature_size', 127)

        # Edge feature size
        self.edge_feature_size = self.config.get('edge_feature_size', 12)

        # Hidden size
        self.hidden_size = self.config.get('hidden_size', 100)

        self.conv1 = GraphConv(self.node_feature_size, self.hidden_size, allow_zero_in_degree='True')
        self.bn1 = nn.BatchNorm1d(self.hidden_size)
        self.dropout1 = nn.Dropout(0.2)

        self.conv2 = GraphConv(self.hidden_size, self.hidden_size, allow_zero_in_degree='True')
        self.bn2 = nn.BatchNorm1d(self.hidden_size)
        self.dropout2 = nn.Dropout(0.2)

        self.conv3 = GraphConv(self.hidden_size, self.hidden_size, allow_zero_in_degree='True')
        self.bn3 = nn.BatchNorm1d(self.hidden_size)
        self.dropout3 = nn.Dropout(0.2)

        self.conv4 = GraphConv(self.hidden_size, self.num_tasks, allow_zero_in_degree='True')

    # def forward(self, g, in_feat):
    def forward(self, mol_dgl_graph, globals):
        mol_dgl_graph.ndata["v"]= mol_dgl_graph.ndata["v"][:,:self.node_feature_size]
        mol_dgl_graph.edata["e"] = mol_dgl_graph.edata["e"][:,:self.edge_feature_size]

        h = self.conv1(mol_dgl_graph, mol_dgl_graph.ndata["v"])
        h = self.bn1(h)
        h = F.relu(h)
        h = self.dropout1(h)

        h = self.conv2(mol_dgl_graph, h)
        h = self.bn2(h)
        h = F.relu(h)
        h = self.dropout2(h)

        h = self.conv3(mol_dgl_graph, h)
        h = self.bn3(h)
        h = F.relu(h)
        h = self.dropout3(h)

        h = self.conv4(mol_dgl_graph, h)

        mol_dgl_graph.ndata["h"] = h

        return dgl.mean_nodes(mol_dgl_graph, "h")

In [ ]:
model_2 = GCN2(config, global_size, num_tasks)

In [ ]:
train_evaluate(model_2)
test_evaluate(model_2)

Save checkpoint
Epoch: 1/100 | Training Loss: 0.842 | Valid Score: 0.415
 
Epoch: 1/100 | Best Valid Score Until Now: 0.415 

Save checkpoint
Epoch: 2/100 | Training Loss: 0.738 | Valid Score: 0.693
 
Epoch: 2/100 | Best Valid Score Until Now: 0.693 

Save checkpoint
Epoch: 3/100 | Training Loss: 0.647 | Valid Score: 0.726
 
Epoch: 3/100 | Best Valid Score Until Now: 0.726 

Save checkpoint
Epoch: 4/100 | Training Loss: 0.596 | Valid Score: 0.743
 
Epoch: 4/100 | Best Valid Score Until Now: 0.743 

Save checkpoint
Epoch: 5/100 | Training Loss: 0.570 | Valid Score: 0.744
 
Epoch: 5/100 | Best Valid Score Until Now: 0.744 

Save checkpoint
Epoch: 6/100 | Training Loss: 0.531 | Valid Score: 0.754
 
Epoch: 6/100 | Best Valid Score Until Now: 0.754 

Save checkpoint
Epoch: 7/100 | Training Loss: 0.511 | Valid Score: 0.758
 
Epoch: 7/100 | Best Valid Score Until Now: 0.758 

Save checkpoint
Epoch: 8/100 | Training Loss: 0.501 | Valid Score: 0.765
 
Epoch: 8/100 | Best Valid Score Until Now: 

#### GraphSAGE

In [ ]:
class SAGEConv(nn.Module):
    def __init__(self, in_feat, out_feat):
        super(SAGEConv, self).__init__()
        # A linear submodule for projecting the input and neighbor feature to the output.
        self.linear = nn.Linear(in_feat * 2, out_feat)

    def forward(self, g, h):
        with g.local_scope():
            g.ndata["h"] = h
            # update_all is a message passing API.
            g.update_all(
                message_func=fn.copy_u("h", "m"),
                reduce_func=fn.mean("m", "h_N"),
            )
            h_N = g.ndata["h_N"]
            h_total = torch.cat([h, h_N], dim=1)
            return self.linear(h_total)

In [ ]:
class GraphSAGE1(nn.Module):
    def __init__(self, config, global_size = 200, num_tasks = 1):
        super().__init__()
        self.config = config
        self.num_tasks = num_tasks

        # Node feature size
        self.node_feature_size = self.config.get('node_feature_size', 127)

        # Edge feature size
        self.edge_feature_size = self.config.get('edge_feature_size', 12)

        # Hidden size
        self.hidden_size = self.config.get('hidden_size', 100)

        self.conv1 = SAGEConv(self.node_feature_size, self.hidden_size)
        self.conv2 = SAGEConv(self.hidden_size, self.num_tasks)

    def forward(self, mol_dgl_graph, globals):
        mol_dgl_graph.ndata["v"]= mol_dgl_graph.ndata["v"][:,:self.node_feature_size]
        mol_dgl_graph.edata["e"] = mol_dgl_graph.edata["e"][:,:self.edge_feature_size]
        h = self.conv1(mol_dgl_graph, mol_dgl_graph.ndata["v"])
        h = F.relu(h)
        h = self.conv2(mol_dgl_graph, h)
        mol_dgl_graph.ndata["h"] = h
        return dgl.mean_nodes(mol_dgl_graph, "h")

In [ ]:
model_3 = GraphSAGE1(config, global_size, num_tasks)

In [ ]:
train_evaluate(model_3)
test_evaluate(model_3)

Save checkpoint
Epoch: 1/100 | Training Loss: 0.679 | Valid Score: 0.262
 
Epoch: 1/100 | Best Valid Score Until Now: 0.262 

Save checkpoint
Epoch: 2/100 | Training Loss: 0.647 | Valid Score: 0.281
 
Epoch: 2/100 | Best Valid Score Until Now: 0.281 

Save checkpoint
Epoch: 3/100 | Training Loss: 0.625 | Valid Score: 0.292
 
Epoch: 3/100 | Best Valid Score Until Now: 0.292 

Save checkpoint
Epoch: 4/100 | Training Loss: 0.609 | Valid Score: 0.304
 
Epoch: 4/100 | Best Valid Score Until Now: 0.304 

Save checkpoint
Epoch: 5/100 | Training Loss: 0.600 | Valid Score: 0.313
 
Epoch: 5/100 | Best Valid Score Until Now: 0.313 

Save checkpoint
Epoch: 6/100 | Training Loss: 0.594 | Valid Score: 0.323
 
Epoch: 6/100 | Best Valid Score Until Now: 0.323 

Save checkpoint
Epoch: 7/100 | Training Loss: 0.586 | Valid Score: 0.339
 
Epoch: 7/100 | Best Valid Score Until Now: 0.339 

Save checkpoint
Epoch: 8/100 | Training Loss: 0.582 | Valid Score: 0.357
 
Epoch: 8/100 | Best Valid Score Until Now: 

#### GrapjSAGE with 4 layers and batch normalization and dropouts.

In [ ]:
class GraphSAGE2(nn.Module):
    def __init__(self, config, global_size = 200, num_tasks = 1):
        super().__init__()
        self.config = config
        self.num_tasks = num_tasks

        # Node feature size
        self.node_feature_size = self.config.get('node_feature_size', 127)

        # Edge feature size
        self.edge_feature_size = self.config.get('edge_feature_size', 12)

        # Hidden size
        self.hidden_size = self.config.get('hidden_size', 100)

        self.conv1 = SAGEConv(self.node_feature_size, self.hidden_size)
        self.bn1 = nn.BatchNorm1d(self.hidden_size)
        self.dropout1 = nn.Dropout(0.2)

        self.conv2 = SAGEConv(self.hidden_size, self.hidden_size)
        self.bn2 = nn.BatchNorm1d(self.hidden_size)
        self.dropout2 = nn.Dropout(0.2)

        self.conv3 = SAGEConv(self.hidden_size, self.hidden_size)
        self.bn3 = nn.BatchNorm1d(self.hidden_size)
        self.dropout3 = nn.Dropout(0.2)

        self.conv4 = SAGEConv(self.hidden_size, self.num_tasks)

    def forward(self, mol_dgl_graph, globals):
        mol_dgl_graph.ndata["v"]= mol_dgl_graph.ndata["v"][:,:self.node_feature_size]
        mol_dgl_graph.edata["e"] = mol_dgl_graph.edata["e"][:,:self.edge_feature_size]

        h = self.conv1(mol_dgl_graph, mol_dgl_graph.ndata["v"])
        h = self.bn1(h)
        h = F.relu(h)
        h = self.dropout1(h)

        h = self.conv2(mol_dgl_graph, h)
        h = self.bn2(h)
        h = F.relu(h)
        h = self.dropout2(h)

        h = self.conv3(mol_dgl_graph, h)
        h = self.bn3(h)
        h = F.relu(h)
        h = self.dropout3(h)

        h = self.conv4(mol_dgl_graph, h)

        mol_dgl_graph.ndata["h"] = h

        return dgl.mean_nodes(mol_dgl_graph, "h")

In [ ]:
model_4 = GraphSAGE2(config, global_size, num_tasks)

In [ ]:
train_evaluate(model_4)
test_evaluate(model_4)

Save checkpoint
Epoch: 1/100 | Training Loss: 0.313 | Valid Score: 0.844
 
Epoch: 1/100 | Best Valid Score Until Now: 0.844 

Patience 1
Epoch: 2/100 | Training Loss: 0.313 | Valid Score: 0.842
 
Epoch: 2/100 | Best Valid Score Until Now: 0.844 

Save checkpoint
Epoch: 3/100 | Training Loss: 0.305 | Valid Score: 0.849
 
Epoch: 3/100 | Best Valid Score Until Now: 0.849 

Save checkpoint
Epoch: 4/100 | Training Loss: 0.297 | Valid Score: 0.852
 
Epoch: 4/100 | Best Valid Score Until Now: 0.852 

Patience 1
Epoch: 5/100 | Training Loss: 0.305 | Valid Score: 0.846
 
Epoch: 5/100 | Best Valid Score Until Now: 0.852 

Patience 2
Epoch: 6/100 | Training Loss: 0.307 | Valid Score: 0.843
 
Epoch: 6/100 | Best Valid Score Until Now: 0.852 

Patience 3
Epoch: 7/100 | Training Loss: 0.305 | Valid Score: 0.847
 
Epoch: 7/100 | Best Valid Score Until Now: 0.852 

Patience 4
Epoch: 8/100 | Training Loss: 0.304 | Valid Score: 0.847
 
Epoch: 8/100 | Best Valid Score Until Now: 0.852 

Patience 5
Epoch:

#### Custom GNN

In [27]:
class CustomGraphConv(nn.Module):
    def __init__(self, in_feat, out_feat):
        super(CustomGraphConv, self).__init__()
        # A linear submodule for projecting the input and neighbor feature to the output.
        self.linear = nn.Linear(in_feat * 2, out_feat)

    def forward(self, g, h):
        with g.local_scope():
            g.ndata["h"] = h
            # update_all is a message passing API.
            g.update_all(
                message_func=fn.u_mul_v("h", "h", "m"),
                reduce_func=fn.sum("m", "h_N"),
            )
            h_N = g.ndata["h_N"]
            h_total = torch.cat([h, h_N], dim=1)
            return self.linear(h_total)

In [28]:
class GNN(nn.Module):
    def __init__(self, config, global_size = 200, num_tasks = 1):
        super().__init__()
        self.config = config
        self.num_tasks = num_tasks

        # Node feature size
        self.node_feature_size = self.config.get('node_feature_size', 127)

        # Edge feature size
        self.edge_feature_size = self.config.get('edge_feature_size', 12)

        # Hidden size
        self.hidden_size = self.config.get('hidden_size', 100)

        self.conv1 = CustomGraphConv(self.node_feature_size, self.hidden_size)
        self.bn1 = nn.BatchNorm1d(self.hidden_size)
        self.dropout1 = nn.Dropout(0.2)

        self.conv2 = CustomGraphConv(self.hidden_size, self.hidden_size)
        self.bn2 = nn.BatchNorm1d(self.hidden_size)
        self.dropout2 = nn.Dropout(0.2)

        self.conv3 = CustomGraphConv(self.hidden_size, self.hidden_size)
        self.bn3 = nn.BatchNorm1d(self.hidden_size)
        self.dropout3 = nn.Dropout(0.2)

        self.conv4 = CustomGraphConv(self.hidden_size, self.num_tasks)

    def forward(self, mol_dgl_graph, globals):
        mol_dgl_graph.ndata["v"]= mol_dgl_graph.ndata["v"][:,:self.node_feature_size]
        mol_dgl_graph.edata["e"] = mol_dgl_graph.edata["e"][:,:self.edge_feature_size]

        h = self.conv1(mol_dgl_graph, mol_dgl_graph.ndata["v"])
        h = self.bn1(h)
        h = F.relu(h)
        h = self.dropout1(h)

        h = self.conv2(mol_dgl_graph, h)
        h = self.bn2(h)
        h = F.relu(h)
        h = self.dropout2(h)

        h = self.conv3(mol_dgl_graph, h)
        h = self.bn3(h)
        h = F.relu(h)
        h = self.dropout3(h)

        h = self.conv4(mol_dgl_graph, h)

        mol_dgl_graph.ndata["h"] = h

        return dgl.mean_nodes(mol_dgl_graph, "h")

In [29]:
model_5 = GNN(config, global_size, num_tasks)

In [30]:
train_evaluate(model_5)
test_evaluate(model_5)

Save checkpoint
Epoch: 1/100 | Training Loss: 0.777 | Valid Score: 0.654
 
Epoch: 1/100 | Best Valid Score Until Now: 0.654 

Save checkpoint
Epoch: 2/100 | Training Loss: 0.800 | Valid Score: 0.692
 
Epoch: 2/100 | Best Valid Score Until Now: 0.692 

Save checkpoint
Epoch: 3/100 | Training Loss: 0.708 | Valid Score: 0.714
 
Epoch: 3/100 | Best Valid Score Until Now: 0.714 

Patience 1
Epoch: 4/100 | Training Loss: 0.703 | Valid Score: 0.711
 
Epoch: 4/100 | Best Valid Score Until Now: 0.714 

Save checkpoint
Epoch: 5/100 | Training Loss: 0.732 | Valid Score: 0.756
 
Epoch: 5/100 | Best Valid Score Until Now: 0.756 

Save checkpoint
Epoch: 6/100 | Training Loss: 0.705 | Valid Score: 0.794
 
Epoch: 6/100 | Best Valid Score Until Now: 0.794 

Patience 1
Epoch: 7/100 | Training Loss: 0.707 | Valid Score: 0.750
 
Epoch: 7/100 | Best Valid Score Until Now: 0.794 

Patience 2
Epoch: 8/100 | Training Loss: 0.676 | Valid Score: 0.721
 
Epoch: 8/100 | Best Valid Score Until Now: 0.794 

Patienc